In [1]:
import requests
import json
import time
from datetime import datetime, date
from hashlib import sha256
import jwt
import re
import copy
BOOKING_URL = "https://cdn-api.co-vin.in/api/v2/appointment/schedule"
URL_PINCODE = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/findByPin"
BENEFICIARIES_URL = "https://cdn-api.co-vin.in/api/v2/appointment/beneficiaries"
CALENDAR_URL_DISTRICT = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByDistrict"
CALENDAR_URL_PINCODE = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByPin"
CAPTCHA_URL = "https://cdn-api.co-vin.in/api/v2/auth/getRecaptcha"
OTP_URL = "https://cdn-api.co-vin.in/api/v2/auth/generateMobileOTP"
VALIDATE_OTP = "https://cdn-api.co-vin.in/api/v2/auth/validateMobileOtp"

base_request_header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
    'origin': 'https://selfregistration.cowin.gov.in/',
    'referer': 'https://selfregistration.cowin.gov.in/'
}

# Generate an otp


def generate_otp(mobile, header):
    data = {
        "mobile": mobile,
        # Some sort of secret key
        "secret": "U2FsdGVkX1+z/4Nr9nta+2DrVJSv7KS6VoQUSQ1ZXYDx/CJUkWxFYG6P3iM/VW+6jLQ9RDQVzp/RcZ8kbT41xw=="
    }
    print(f"Requesting OTP with mobile number {mobile}..")
    txnid = requests.post(url=OTP_URL, json=data, headers=header)
    if txnid.status_code == 200:
        txnid = txnid.json()["txnId"]
        return txnid
    else:
        return None


# Verify that otp
def verify_otp(txnid, header):
    # t_end = time.time() + 60 * 3  # try to read OTP for atmost 3 minutes
    # while time.time() < t_end:
    #     response = requests.get(storage_url)
    #     if response.status_code == 200:
    #         print("OTP SMS is:" + response.text)

    #         OTP = response.text
    #         OTP = re.findall("[0-9]{6}",OTP)[0]
    #         if not OTP:
    #             time.sleep(5)
    #             continue
    #         break
    #     else:
    #         # Hope it won't 500 a little later
    #         print("error fetching OTP API:" + response.text)
    #         time.sleep(5)

    # if not OTP:
    #     return None
    otp = int(input())
    data = {"otp": sha256(str(otp).encode(
        'utf-8')).hexdigest(), "txnId": txnid}
    token = requests.post(url=VALIDATE_OTP, json=data, headers=header)
    if token.status_code == 200:
        return token.json()["token"]
    else:
        return None


def is_token_valid(token):
    payload = jwt.decode(token, options={"verify_signature": False})
    remaining_seconds = payload['iat'] + 600 - int(time.time())
    if remaining_seconds <= 60:
        print("Token is about to expire in next 1 min ...")
    if remaining_seconds <= 1*30:  # 30 secs early before expiry for clock issues
        return False
    return True

# Create header with the authorization token (Bearer Token)


def create_header(auth_token):
    header = copy.deepcopy(base_request_header)
    header["Authorization"] = f"Bearer {auth_token}"
    return header


# get the today's date and make a tuple in format of date,month,year
def currrent_date():
    date = time.localtime()
    return (date.tm_mday, date.tm_mon, date.tm_year)


def next_date(n):
    from datetime import timedelta, date
    date_t = date.today() + timedelta(n)
    return date_t.day, date_t.month, date_t.year


# Function to fetch data from the cowin api
def fetch_data_calendar(pincode, date, header, vaccine_type):
    date_string = '-'.join(str(x) for x in date)
    url = CALENDAR_URL_PINCODE
    if vaccine_type != " ":
        data = {"pincode": pincode, "date": date_string, "vaccine": vaccine_type}
    else:
        data = {"pincode": pincode, "date": date_string}
    result = requests.get(url, params=data, headers=header)
    return result

# Fetch data in realtime


def fetch_data_date(pincode, date, header):
    date_string = '-'.join(str(x) for x in date)
    url = URL_PINCODE
    data = {"pincode": pincode, "date": date_string}
    result = requests.get(url, params=data, headers=header)
    return result

# Fetch data by calendar district


def fetch_data_calendar_district(district_id, date, header):
    date_string = '-'.join(str(x) for x in date)
    url = CALENDAR_URL_DISTRICT
    data = {"district_id": district_id, "date": date_string}
    result = requests.get(url, params=data, headers=header)
    return result

# See all the centers present in that pincode for a particular date


def centers_by_date(data):
    centers_data = []
    for x in data["sessions"]:
        centers_data.append(
            (x["center_id"], x["name"], x["vaccine"], x["available_capacity"]))
    return centers_data

# See all the available centers in next 7 dats


def centers_by_calendar(data):
    centers_data = []
    for x in data["centers"]:
        for j in x["sessions"]:
            centers_data.append(
                (x["center_id"], x["name"],  j["vaccine"], j["available_capacity_dose1"], j["available_capacity_dose2"], j["date"]))
    return centers_data


# Vaccine centers for dose 1 regardless of vaccine name but with regard of age
def dose_1_date(data, min_age):
    dose_1_data = []
    for x in data["sessions"]:
        if x["available_capacity"] > 0 and x["min_age_limit"] == min_age:
            dose_1_data.append(
                (x["center_id"], x["name"], x["vaccine"], x["available_capacity_dose1"]))
    return dose_1_data


# Vaccine centers for dose 2 with regard to vaccine type and age limit


def dose_2_date(data, min_age, vaccine_type):
    dose_2_data = []
    for x in data["sessions"]:
        if(x["min_age_limit"] == min_age and vaccine_type != "" and x["vaccine"] == vaccine_type and x["available_capacity_dose2"] > 0):
            dose_2_data.append(
                (x["center_id"], x["name"], x["vaccine"], x["available_capacity_dose2"]))
    return dose_2_data

# Vaccien centers for dose 2 for next 7 days


def calendar_data(data, min_age, option):
    cal_data = []
    for x in data["centers"]:
        for j in x["sessions"]:
            if(j["min_age_limit"] == min_age):
                if option == 2 and j["available_capacity_dose2"] > 0:
                    cal_data.append(
                        (x["center_id"], x["name"],  j["vaccine"], j["available_capacity_dose2"], j["date"]))
                elif option == 1 and j["available_capacity_dose1"] > 0:
                    cal_data.append(
                        (x["center_id"], x["name"],  j["vaccine"], j["available_capacity_dose1"], j["date"]))

    return cal_data


def fetch_beneficiaries(header):
    return requests.get(BENEFICIARIES_URL, headers=header)


def initialize_token(mobile):
    txnid = None
    while txnid is None:
        try:
            txnid = generate_otp(mobile, base_request_header)
            print("Otp generated:")
        except Exception as e:
            print(str(e))
            print('OTP Retrying in 5 seconds')
            time.sleep(5)

    auth_token = None
    while auth_token is None:
        try:
            auth_token = verify_otp(txnid, base_request_header)
        except Exception as e:
            print(str(e))
            print('OTP Retrying in 5 seconds')
            time.sleep(5)

    header = create_header(auth_token)
    return header, auth_token


In [2]:
def slot_finder(option, pincode, vaccine, min_age_limit, auth_token, header, date):
    if option == 1:
        result = fetch_data_calendar(pincode, date, header, "")

    elif option == 2:
        result = fetch_data_calendar(pincode, date, header, vaccine)

    if result.status_code == 200:
        result = result.json()
    received_data = calendar_data(result, min_age_limit,option)
    if received_data != []:
        for x in received_data:
            print(x)
        return True
    if received_data==[]:
        return False


In [3]:
def main():
    mobile = int(input("Mobile Number"))
    pincode = input("Pincode")
    vaccine = input("Name of vaccine (Leave blank if no preferrence)").upper()
    min_age_limit = int(input("Min age"))
    option = int(input("Options :\n 1. Dose 1 \n2.Dose 2"))
    delay = int(input("Delay time greater than 5(default = 5)"))
    if(delay<5):
        delay=5
    date = currrent_date()
    header, auth_token = initialize_token(mobile)
    print("Initialized token succesfully")
    istrue = True
    while istrue:
        print(time.ctime(), "\n")
        if time.localtime().tm_mday != date[0]:
            date = currrent_date()
        try:
            is_valid = is_token_valid(auth_token)
            print(is_valid)
            if not is_valid:
                header, auth_token = initialize_token(mobile)
                print("slot")
            else:
                x = slot_finder(option, pincode, vaccine,min_age_limit, auth_token, header, date)
                if x:
                    istrue = False
        except Exception as e:
            print(str(e))
        time.sleep(delay)

In [ ]:
fetch_data_calendar_district(333,(31,5,2021),header)

In [ ]:
# res = fetch_data_date(485001,(31,5,2021),header)  #fetch data by a particular date
res2 = fetch_data_calendar(485001,(1,6,2021),header,"COVAXIN")

In [ ]:
res2

In [4]:
main()

Requesting OTP with mobile number 8817141845..
Otp generated:
Initialized token succesfully
Tue Jun  1 13:54:27 2021 

True
Tue Jun  1 13:54:32 2021 

True
Tue Jun  1 13:54:37 2021 

True
Tue Jun  1 13:54:43 2021 

True
Tue Jun  1 13:54:48 2021 

True
Tue Jun  1 13:54:54 2021 

True
Tue Jun  1 13:55:00 2021 

True
Tue Jun  1 13:55:05 2021 

True
Tue Jun  1 13:55:10 2021 

True
Tue Jun  1 13:55:16 2021 

True
Tue Jun  1 13:55:21 2021 

True
Tue Jun  1 13:55:27 2021 

True
Tue Jun  1 13:55:32 2021 

True
Tue Jun  1 13:55:38 2021 

True
Tue Jun  1 13:55:43 2021 

True
Tue Jun  1 13:55:48 2021 

True
Tue Jun  1 13:55:54 2021 

True
Tue Jun  1 13:55:59 2021 

True
Tue Jun  1 13:56:05 2021 

True
Tue Jun  1 13:56:10 2021 

True
Tue Jun  1 13:56:16 2021 

True
Tue Jun  1 13:56:21 2021 

True
Tue Jun  1 13:56:27 2021 

True
Tue Jun  1 13:56:32 2021 

True
Tue Jun  1 13:56:38 2021 

True
Tue Jun  1 13:56:43 2021 

True
Tue Jun  1 13:56:49 2021 

True
Tue Jun  1 13:56:54 2021 

True
Tue Jun  1 1

KeyboardInterrupt: 

In [ ]:
token=generate_otp(8817141845,base_request_header)

In [ ]:
token=token.json()["txnId"]

In [ ]:
token

In [ ]:
auth_token=verify_otp(310040,token,base_request_header)

In [ ]:
auth_token.text

In [ ]:

header = create_header(auth)

In [ ]:
header

In [ ]:
import jwt
payload = jwt.decode(auth, options={"verify_signature": False})
payload

In [ ]:
res = fetch_data_calender(485001,(26,5,2021),header)  #fetch data by a particular date

In [ ]:
requests.get(url="https://cdn-api.co-vin.in/api/v2/registration/certificate/public/download?beneficiary_reference_id=39711767954990",headers=header)

In [ ]:
is_token_valid(auth_token)

In [ ]:
header,auth=initialize_token()


In [ ]:
auth